# ニューラルネットワークによるアヤメの分類(chainer)

In [ ]:
import numpy as np
np.random.seed(3)

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training, datasets
from chainer.training import extensions

In [ ]:
# データの読込
from sklearn.datasets import load_iris
iris = load_iris()

# 入力変数と説明変数の切り分け
X = iris.data.astype(np.float32)
Y = iris.target.flatten()

# 訓練データと検証データに分割(70%を訓練用に使用)
dataset = list(zip(X, Y))
n_train = int(len(dataset) * 0.7)
train, test = chainer.datasets.split_dataset_random(dataset, n_train, seed=1)

In [ ]:
# モデルの構成
class NN(chainer.Chain):

    def __init__(self):
        super().__init__()
        with self.init_scope():
            self.fc1 = L.Linear(4, 30) 
            self.fc2 = L.Linear(30, 20) 
            self.fc3 = L.Linear(20, 3)  

    def __call__(self, x):
        u1 = self.fc1(x)
        z1 = F.relu(u1)
        u2 = self.fc2(z1)
        z2 = F.relu(u2)
        u3 = self.fc3(z2)
        
        return u3
    
nn = NN()
model = L.Classifier(nn)
model.lossfun

In [ ]:
# Optimizerの構成 (バックプロパゲーション)
optimizer = chainer.optimizers.SGD()
optimizer.setup(model)
optimizer.lr

In [ ]:
# Iterator,Updaterの構成 (学習サイクルなど)
batchsize =50

train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter  = chainer.iterators.SerialIterator(test,  batchsize, repeat=False, shuffle=False)
updater = training.StandardUpdater(train_iter, optimizer, device=-1)

In [ ]:
# ログの生成と学習の実行
epoch = 100

trainer = training.Trainer(updater, (epoch, 'epoch'), out='chainer_log/iris/')

trainer.extend(extensions.dump_graph('main/loss'))
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport(trigger=(1, 'epoch')))
trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy', 'main/loss', 'validation/main/loss', 'elapsed_time']), trigger=(1, 'epoch'))
trainer.extend(extensions.dump_graph(root_name="main/loss", out_name="cg.dot"))

trainer.run()

In [ ]:
# モデル精度の表示(検証データ)
print(model.accuracy.data)

In [ ]:
# ロス率の表示(検証データ)
print(model.loss.data)

In [ ]:
# モデル精度の可視化
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 15,5
import json
import pandas as pd
import matplotlib.pyplot as plt
with open('chainer_log/iris/log') as f:
    logs = json.load(f)
    results = pd.DataFrame(logs)

results[['main/accuracy', 'validation/main/accuracy']].plot()
results[['main/loss', 'validation/main/loss']].plot()

In [ ]:
# 元データの150件目(X[149])でクラスごとの分類確率を表示[Setosa, Versicolor, Virginica]
y = model.predictor(X[[ 149 ]])
print (F.softmax(y).data)

In [ ]:
# 元データの150件目(X[149])で分類結果を表示[0=Setosa,1=Versicolor, 2=Virginica]
y = model.predictor(X[[ 149 ]])
print (F.softmax(y).data.argmax(axis=1))

In [ ]:
# 元データの150件目(Y[149])の正解を表示(0=Setosa,1=Versicolor, 2=Virginica)
print ([Y[149]])

In [ ]:
# 参考: 手動で値設定する方法
y = model.predictor(np.array([[ 5.9, 3. , 5.1, 1.8 ]]).astype(np.float32))
print (F.softmax(y).data.argmax(axis=1))

In [ ]:
# 参考: モデル構造の可視化
import pydotplus
import matplotlib.pyplot as plt
from pylab import rcParams

rcParams['figure.figsize'] = 25,30
graph = pydotplus.graph_from_dot_file("chainer_log/iris/cg.dot") 
graph.write_png("chainer_log/iris/cg.png") 
img = plt.imread("chainer_log/iris/cg.png")
plt.imshow(img)
plt.show()   